In [1]:
import requests
import pandas as pd
from sqlalchemy import create_engine
from tqdm.notebook import tqdm
import time

In [2]:
# Le début des relevés est 2013. 

def get_all_data():
    base_url = "https://hubeau.eaufrance.fr/api/v1/vente_achat_phyto/achats/substances"
    params = {
        "type_territoire": "Département",
        "annee_min": 2012,
        "annee_max": 2026,
        "size": 20000,
        "page": 1
    }

    all_data = []

    print(" Accès à l'API en cours... Veuillez patienter.")
    
    start_time = time.time()
    
    while True:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data_page = response.json().get("data", [])
        
        if not data_page:
            break
        
        all_data.extend(data_page)
        params["page"] += 1

        
    end_time = time.time()
    duration = end_time - start_time

    print(f"Nombre de fichiers récupérés :{len(all_data)}")
    return pd.DataFrame(all_data)


In [3]:
df = get_all_data()

 Accès à l'API en cours... Veuillez patienter.
Nombre de fichiers récupérés :1298846


In [4]:
df.head()

,amm,annee,achat_etranger,classification,classification_mention,code_cas,code_substance,code_territoire,fonction,libelle_substance,libelle_territoire,quantite,type_territoire,uri_substance,uri_territoire
0,2210071,2023,Non,Env B,None,1918-00-9,1480,16,Herbicide,dicamba,CHARENTE,32.00,Département,http://id.eaufrance.fr/par/1480,http://id.eaufrance.fr/dep/16
1,9900340,2023,Non,Env A,None,137641-05-5,5665,49,Herbicide,picolinafen,MAINE-ET-LOIRE,16.16,Département,http://id.eaufrance.fr/par/5665,http://id.eaufrance.fr/dep/49
2,9800182,2023,Non,Env A,None,87392-12-9,8070,67,Herbicide,s-metolachlore,BAS-RHIN,15163.68,Département,http://id.eaufrance.fr/par/8070,http://id.eaufrance.fr/dep/67
3,2110191,2023,Non,CMR,None,119446-68-3,1905,60,Fongicide,difenoconazole,OISE,3527.60,Département,http://id.eaufrance.fr/par/1905,http://id.eaufrance.fr/dep/60
4,2090153,2023,Non,Env A,None,142469-14-5,7087,25,Herbicide,tritosulfuron,DOUBS,78.00,Département,http://id.eaufrance.fr/par/7087,http://id.eaufrance.fr/dep/25


In [5]:
# Souhait de ne conserver que les substances étant classifiées CMR. Avant 2018, il était écrit T, T+, CMR
# et à partir de 2018, juste CMR. Je choisi donc de filtrer mon dataframe pour ne conserver que ces valeurs.
df_cmr = df[df["classification"].isin(["CMR", "T, T+, CMR"])]


In [6]:
df_cmr.head()

,amm,annee,achat_etranger,classification,classification_mention,code_cas,code_substance,code_territoire,fonction,libelle_substance,libelle_territoire,quantite,type_territoire,uri_substance,uri_territoire
3,2110191,2023,Non,CMR,None,119446-68-3,1905,60,Fongicide,difenoconazole,OISE,3527.60,Département,http://id.eaufrance.fr/par/1905,http://id.eaufrance.fr/dep/60
7,8700020,2023,Non,CMR,Exclusion,61213-25-0,1675,30,Herbicide,flurochloridone,GARD,131.25,Département,http://id.eaufrance.fr/par/1675,http://id.eaufrance.fr/dep/30
10,2160198,2023,Non,CMR,None,107534-96-3,1694,53,Fongicide,tebuconazole,MAYENNE,11.25,Département,http://id.eaufrance.fr/par/1694,http://id.eaufrance.fr/dep/53
15,7400755,2023,Non,CMR,None,86-87-3,5584,38,Régulateur de croissance,acide alpha naphtylacetique (ana),ISERE,6.05,Département,http://id.eaufrance.fr/par/5584,http://id.eaufrance.fr/dep/38
17,2140110,2023,Non,CMR,None,67129-08-2,1670,69,Herbicide,metazachlore,RHONE,2.00,Département,http://id.eaufrance.fr/par/1670,http://id.eaufrance.fr/dep/69


In [7]:
# Conservation des colonnes utiles pour la base de données. 
df_cmr = df_cmr[["amm", "annee", "classification_mention", "code_cas", "code_substance", "code_territoire", "fonction", "libelle_substance", "quantite"]]

In [8]:
df_cmr = df_cmr.reset_index(drop=True)

In [9]:
# Je vais maintenant supprimer les départements ayant le code territoire 0, 
# représentant une erreur dans la saisie. 
len(df_cmr)

276855

In [10]:
(df_cmr["code_territoire"] == "0").sum()

np.int64(0)

In [11]:
df_cmr = df_cmr[df_cmr["code_territoire"] != "0"]

In [12]:
len(df_cmr)

276855

In [13]:
df_cmr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276855 entries, 0 to 276854
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   amm                     276855 non-null  object 
 1   annee                   276855 non-null  int64  
 2   classification_mention  183267 non-null  object 
 3   code_cas                276855 non-null  object 
 4   code_substance          271724 non-null  object 
 5   code_territoire         276855 non-null  object 
 6   fonction                276855 non-null  object 
 7   libelle_substance       276855 non-null  object 
 8   quantite                276855 non-null  float64
dtypes: float64(1), int64(1), object(7)
memory usage: 19.0+ MB


In [14]:
# Je vais maintenant convertir les codes en int et changer les noms pour
# les rendre plus compréhensibles

colonnes_entiers = ["amm", "code_substance"] 
for col in colonnes_entiers:
    df_cmr[col] = pd.to_numeric(df_cmr[col], errors="coerce").astype("Int64")


In [15]:
df_cmr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276855 entries, 0 to 276854
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   amm                     276855 non-null  Int64  
 1   annee                   276855 non-null  int64  
 2   classification_mention  183267 non-null  object 
 3   code_cas                276855 non-null  object 
 4   code_substance          271724 non-null  Int64  
 5   code_territoire         276855 non-null  object 
 6   fonction                276855 non-null  object 
 7   libelle_substance       276855 non-null  object 
 8   quantite                276855 non-null  float64
dtypes: Int64(2), float64(1), int64(1), object(5)
memory usage: 19.5+ MB


In [16]:
df_cmr = df_cmr.rename(columns={
    "code_territoire": "num_département",
    "libelle_substance" : "nom_substance",
    "quantite": "quantite_en_kg",
})

df_cmr.head()

,amm,annee,classification_mention,code_cas,code_substance,num_département,fonction,nom_substance,quantite_en_kg
0,2110191,2023,None,119446-68-3,1905,60,Fongicide,difenoconazole,3527.60
1,8700020,2023,Exclusion,61213-25-0,1675,30,Herbicide,flurochloridone,131.25
2,2160198,2023,None,107534-96-3,1694,53,Fongicide,tebuconazole,11.25
3,7400755,2023,None,86-87-3,5584,38,Régulateur de croissance,acide alpha naphtylacetique (ana),6.05
4,2140110,2023,None,67129-08-2,1670,69,Herbicide,metazachlore,2.00


Une fois les réglages effectués, on passe au script python qui va enregistrer en base de données. 
Lancer le script Insertion_data.py

In [17]:
for col in df_cmr.select_dtypes(include="object").columns:
    max_len = df_cmr[col].astype(str).str.len().max()
    print(f"{col} : VARCHAR({max_len})")


classification_mention : VARCHAR(12)
code_cas : VARCHAR(11)
num_département : VARCHAR(3)
fonction : VARCHAR(56)
nom_substance : VARCHAR(66)


In [18]:
for col in df_cmr.select_dtypes(include="int").columns:
    max_val = df_cmr[col].max()
    min_val = df_cmr[col].min()
    max_digits = max(len(str(abs(max_val))), len(str(abs(min_val))))
    print(f"{col} : INTEGER (min={min_val}, max={max_val}, digits={max_digits})")


amm : INTEGER (min=2000003, max=9900446, digits=7)
annee : INTEGER (min=2013, max=2023, digits=4)
code_substance : INTEGER (min=1083, max=8744, digits=4)


In [19]:
for col in df_cmr.select_dtypes(include="float").columns:
    max_val = df_cmr[col].max()
    min_val = df_cmr[col].min()

    def count_digits(val):
        if pd.isna(val):
            return (0, 0)
        parts = f"{val:.10f}".rstrip("0").split(".")
        return (len(parts[0]), len(parts[1]) if len(parts) > 1 else 0)

    entiers, decimales = zip(*df_cmr[col].dropna().map(count_digits))

    print(f"{col} : NUMERIC({max(entiers)+max(decimales)}, {max(decimales)}) "
          f"(min={min_val}, max={max_val})")


quantite_en_kg : NUMERIC(12, 6) (min=0.0, max=364377.24)


In [20]:
df_cmr["num_département"].value_counts()

num_département
17     4427
51     4364
59     4165
02     4064
10     4039
       ... 
974     295
972     107
971     104
973      65
976      37
Name: count, Length: 102, dtype: int64